In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from dppy.multivariate_jacobi_ope import MultivariateJacobiOPE
import scipy.stats as stats
from scipy.stats import multivariate_normal
from scipy.sparse.linalg.eigen.arpack import eigsh as largest_eigsh
from scipy.io import savemat
from scipy.io import loadmat
from scipy.optimize import minimize
from dppy.finite_dpps import FiniteDPP
import math
import array
from random import seed
from random import randint

In [ ]:
from Data_generation import generate_data_uniform
from Data_generation import generate_data_beta
from Data_generation import generate_data_mixture_Gaussian
from Jacobi_parameter import fit_Jacobi_parameters

In [ ]:
# Function for dividing the data into batches
def get_batches(X,y,batch_size,row,sampleops,i):
    if sampleops.name == 'iid':
        idx = random.sample(range(row), batch_size)
    if sampleops.name == 'dpp':
        idx = sampleops.DPP_list[i]
    idx = np.sort(idx)
    X_new = X[idx,:]
    y_new = y[idx]
    return X_new, y_new, idx

# Function for computing the gradient
def get_gradient(X,y,theta,weight,loss_type,elambda):
    hypothesis = np.dot(X, theta)
    if loss_type == 'linear_regression':
        loss = hypothesis - y
    elif loss_type == 'logistic_regression':
        loss = - y * (1 - 1 / (1 + np.exp(- hypothesis * y)))
    gradient = np.dot(X.T, weight * loss) + elambda * theta
    return gradient

# Function for generating the alternative gradient
def get_gradient_dppway2(X,y,theta,batch_size,loss_type,elambda,sampleops):
    N, d = X.shape
    hypothesis = np.dot(X, theta)
    if loss_type == 'linear_regression':
        loss = hypothesis - y
    elif loss_type == 'logistic_regression':
        loss = - y * (1 - 1 / (1 + np.exp(- hypothesis * y)))
    Xsample = dpp.sample()
    gradient = 0
    for i in range(batch_size):
        xsam = Xsample[i,:]
        tmp = np.dot(np.ones((N,1)),np.reshape(xsam,(1,d))) - X
        weight = np.reshape(sampleops.var.pdf(tmp), (N,1))
        nablahat = np.dot(X.T, weight * loss) / N
        gradient = gradient + nablahat / sampleops.dpp.K(xsam, eval_pointwise=False) / sampleops.dpp.eval_w(xsam)
    gradient = gradient + elambda * theta
    return gradient

# Function for computing the function value
def get_fun_value(X,y,theta,N,loss_type,elambda):
    hypothesis = np.dot(X, theta)
    if loss_type == 'linear_regression':
        loss = hypothesis - y
        fun_value = 0.5 * np.dot(loss.T,loss) / N + 0.5 * elambda * np.dot(theta.T,theta)
    elif loss_type == 'logistic_regression':
        fun_value = np.sum(np.log(1 + np.exp(-hypothesis * y))) / N + 0.5 * elambda * np.dot(theta.T,theta)
    return fun_value

# Function for generating the DPP kernel for first way of gradient estimation
def generate_DPP_kernel(X,N,p,dpp,gammatildeX):
    Kq = dpp.K(X, eval_pointwise=False)
    qX = dpp.eval_w(X)
    D = np.diag(np.sqrt(np.divide(qX, gammatildeX)))
    Ktilde = 1. / N * D @ Kq @ D
    evals_large_sparse, evecs_large_sparse = largest_eigsh(Ktilde, p, which='LM')
    evals_large_sparse = np.ones(p)
    Ktilde = np.dot(evecs_large_sparse,evecs_large_sparse.T)
    diagKtilde = np.diag(Ktilde)
    return evals_large_sparse, evecs_large_sparse, diagKtilde

# Function for sampling the finite DPP
def generate_DPP_list_of_samples(eig_vals, eig_vecs, maxit):
    DPP = FiniteDPP(kernel_type='correlation',projection=True,
                    **{'K_eig_dec': (eig_vals, eig_vecs)})
    for _ in range(maxit):
        DPP.sample_exact(mode='GS')
    return DPP.list_of_samples

In [ ]:
# Mini-Batch SGD
def MiniBatchSGD(X,y,theta,loss_type,elambda,batch_size,maxiter,sampleops,thetastar=np.nan):
    N, d = X.shape
    if sampleops.name == 'iid':
        weight = (1 / batch_size) * np.ones((N, 1))
    if sampleops.name == 'dpp':
        weight = np.reshape(1 / sampleops.kernel_diag / N, (N,1))
    loss_total = np.array(maxiter*[[0.0]])
    gradient_total = np.array(maxiter*[[0.0]])
    if ~np.isnan(thetastar).all():
        error = np.array(maxiter*[[0.0]])
    for i in range(maxiter):
        if sampleops.name == 'dpp2':
            gradient = get_gradient_dppway2(X,y,theta,batch_size,loss_type,elambda,sampleops)
        else:
            X_batch, y_batch, idx = get_batches(X,y,batch_size,N,sampleops,i)
            gradient = get_gradient(X_batch,y_batch,theta,weight[idx],loss_type,elambda)
        theta = theta - (1/(i+1)**0.9) * gradient
        loss_total[i] = get_fun_value(X,y,theta,N,loss_type,elambda)
        gradient_total[i] = np.linalg.norm(get_gradient(X,y,theta,(1 / N) * np.ones((N, 1)),loss_type,elambda), 2)
        if ~np.isnan(thetastar).all():
            error[i] = np.linalg.norm(theta - thetastar, 2)
    if np.isnan(thetastar).all():
        return theta, loss_total, gradient_total
    if ~np.isnan(thetastar).all():
        return theta, loss_total, gradient_total, error

In [ ]:
class sample_ops:
    def __init__(self):
        self.name = []

N, d = 1000, 2
losstype = 'linear_regression'

#X, y = generate_data_uniform(N, d)
X ,y = generate_data_mixture_Gaussian(N, d, 2)
Z = np.concatenate((X,y),axis = 1)
        
theta0 = np.array(d*[[0.0]])
lambda_input = 0.1

In [ ]:
if losstype == 'linear_regression':
    inv = np.linalg.inv(np.dot(X.T, X) + N * lambda_input * np.identity(d))
    theta_direct = np.dot(inv, np.dot(X.T, y))
if losstype == 'logistic_regression':
    def objfun(x):
        x = np.reshape(x,(d,1))
        hypothesis = np.dot(X, x)
        fun_value = np.sum(np.log(1 + np.exp(-hypothesis * y))) / N + 0.5 * lambda_input * np.dot(x.T,x)
        fun_value = fun_value[0,0]
        return fun_value
    theta_solve = minimize(objfun, np.reshape(theta0,(d,)), tol=1e-16)
    theta_direct = np.reshape(theta_solve.x,(d,1))
gradient = get_gradient(X,y,theta_direct,(1 / N) * np.ones((N, 1)),losstype,lambda_input)
norm_gradient = np.linalg.norm(gradient, 2)
print('True solution obtained, with norm of gradient = ',norm_gradient)

In [ ]:
p2 = 5
maxit2 = int(5*N/p2)
p3 = 10
maxit3 = int(5*N/p3)
p4 = p2
maxit4 = int(5*N/p4)
p5 = p3
maxit5 = int(5*N/p5)
p6 = p2
maxit6 = int(5*N/p6)
p7 = p3
maxit7 = int(5*N/p7)

trial_total = 1000

ops = sample_ops()
ops.name = 'iid'

## generate DPP kernel and gammatilde
jac_params = fit_Jacobi_parameters(Z)
#jac_params = np.array([[1. / 3, 1. / 2] for _ in range(d+1)])
gammatilde = stats.gaussian_kde(Z.T)
gammatilde.set_bandwidth(bw_method='silverman')
gammatildeZ = gammatilde.evaluate(Z.T)
#gammatildeZ = 1./2**(d+1) * np.ones((N,))
    
for trial_num in range(trial_total):
    ops1 = sample_ops()
    ops1.name = 'iid'
    
    theta_start, loss_total, grad_total, error = MiniBatchSGD(X, y, theta0, loss_type=losstype, elambda=lambda_input,
              batch_size=N, maxiter=1, sampleops=ops, thetastar=theta_direct)
    
    theta2, loss_total2, grad_total2, error2 = MiniBatchSGD(X, y, theta_start, loss_type=losstype, elambda=lambda_input,
                         batch_size=p2, maxiter=maxit2, sampleops=ops1, thetastar=theta_direct)
    
    theta3, loss_total3, grad_total3, error3 = MiniBatchSGD(X, y, theta_start, loss_type=losstype, elambda=lambda_input,
                     batch_size=p3, maxiter=maxit3, sampleops=ops1, thetastar=theta_direct)

    ops2 = sample_ops()
    ops2.name = 'dpp'
    
    dpp = MultivariateJacobiOPE(p4, jac_params)
    eig_vals, eig_vecs, diagKtilde = generate_DPP_kernel(Z,N,p4,dpp,gammatildeZ)
    ops2.kernel_diag = diagKtilde
    ops2.DPP_list = generate_DPP_list_of_samples(eig_vals, eig_vecs, maxit4)
    theta4, loss_total4, grad_total4, error4 = MiniBatchSGD(X, y, theta_start, loss_type=losstype, elambda=lambda_input,
                         batch_size=p4, maxiter=maxit4, sampleops=ops2, thetastar=theta_direct)

    dpp = MultivariateJacobiOPE(p5, jac_params)
    eig_vals, eig_vecs, diagKtilde = generate_DPP_kernel(Z,N,p5,dpp,gammatildeZ)
    ops2.kernel_diag = diagKtilde
    ops2.DPP_list = generate_DPP_list_of_samples(eig_vals, eig_vecs, maxit5)
    theta5, loss_total5, grad_total5, error5 = MiniBatchSGD(X, y, theta_start, loss_type=losstype, elambda=lambda_input,
                         batch_size=p5, maxiter=maxit5, sampleops=ops2, thetastar=theta_direct)
    
    if trial_num == 0:
        loss2 = loss_total2
        loss3 = loss_total3
        loss4 = loss_total4
        loss5 = loss_total5
        grad2 = grad_total2
        grad3 = grad_total3
        grad4 = grad_total4
        grad5 = grad_total5
        err2 = error2
        err3 = error3
        err4 = error4
        err5 = error5
    else:
        loss2 = np.hstack((loss2, loss_total2))
        loss3 = np.hstack((loss3, loss_total3))
        loss4 = np.hstack((loss4, loss_total4))
        loss5 = np.hstack((loss5, loss_total5))
        grad2 = np.hstack((grad2, grad_total2))
        grad3 = np.hstack((grad3, grad_total3))
        grad4 = np.hstack((grad4, grad_total4))
        grad5 = np.hstack((grad5, grad_total5))
        err2 = np.hstack((err2, error2))
        err3 = np.hstack((err3, error3))
        err4 = np.hstack((err4, error4))
        err5 = np.hstack((err5, error5))
    print(trial_num, 'finished')

In [ ]:
start2 = int(0.1*N/p2)
start3 = int(0.1*N/p3)

plt.rcParams.update({'font.size': 20})
plot1 = plt.figure(1)
plt.plot(range(start2*p2,maxit2*p2,p2),loss2.mean(1)[start2:maxit2],'r', label='iid_p5')
plt.plot(range(start3*p3,maxit3*p3,p3),loss3.mean(1)[start3:maxit3],'b', label='iid_p10')
plt.plot(range(start2*p2,maxit2*p2,p2),loss4.mean(1)[start2:maxit2],'r--', label='dpp_p5')
plt.plot(range(start3*p3,maxit3*p3,p3),loss5.mean(1)[start3:maxit3],'b--', label='dpp_p10')
plt.xlabel('budget')
plt.ylabel('function value')
plt.legend(fontsize=20)
plt.show()

plt.rcParams.update({'font.size': 20})
plot2 = plt.figure(2)
plt.plot(range(start2*p2,maxit2*p2,p2),grad2.mean(1)[start2:maxit2],'r', label='iid_p5')
plt.plot(range(start3*p3,maxit3*p3,p3),grad3.mean(1)[start3:maxit3],'b', label='iid_p10')
plt.plot(range(start2*p2,maxit2*p2,p2),grad4.mean(1)[start2:maxit2],'r--', label='dpp_p5')
plt.plot(range(start3*p3,maxit3*p3,p3),grad5.mean(1)[start3:maxit3],'b--', label='dpp_p10')
plt.xlabel('budget')
plt.ylabel('norm of gradient')
plt.legend(fontsize=20)
plt.show()

plt.rcParams.update({'font.size': 20})
plot3 = plt.figure(3)
plt.plot(range(start2*p2,maxit2*p2,p2),err2.mean(1)[start2:maxit2],'r', label='iid_p5')
plt.plot(range(start3*p3,maxit3*p3,p3),err3.mean(1)[start3:maxit3],'b', label='iid_p10')
plt.plot(range(start2*p2,maxit2*p2,p2),err4.mean(1)[start2:maxit2],'r--', label='dpp_p5')
plt.plot(range(start3*p3,maxit3*p3,p3),err5.mean(1)[start3:maxit3],'b--', label='dpp_p10')
plt.xlabel('budget')
plt.ylabel('$||\Theta_t-\Theta_*||_2$')
plt.legend(fontsize=20)
plt.show()

In [ ]:
plt.rcParams.update({'font.size': 20})
plot4 = plt.figure(4)
plt.plot(range(start2*p2,maxit2*p2,p2),grad2.mean(1)[start2:maxit2],'r', label='iid_p5')
plt.plot(range(start3*p3,maxit3*p3,p3),grad3.mean(1)[start3:maxit3],'b', label='iid_p10')
plt.plot(range(start2*p2,maxit2*p2,p2),grad4.mean(1)[start2:maxit2],'r--', label='dpp_p5')
plt.plot(range(start3*p3,maxit3*p3,p3),grad5.mean(1)[start3:maxit3],'b--', label='dpp_p10')
plt.xlabel('budget')
plt.ylabel('norm of gradient')
plt.legend(fontsize=20)
plt.yscale('log')
plt.show()

plt.rcParams.update({'font.size': 20})
plot5 = plt.figure(5)
plt.plot(range(start2*p2,maxit2*p2,p2),err2.mean(1)[start2:maxit2],'r', label='iid_p5')
plt.plot(range(start3*p3,maxit3*p3,p3),err3.mean(1)[start3:maxit3],'b', label='iid_p10')
plt.plot(range(start2*p2,maxit2*p2,p2),err4.mean(1)[start2:maxit2],'r--', label='dpp_p5')
plt.plot(range(start3*p3,maxit3*p3,p3),err5.mean(1)[start3:maxit3],'b--', label='dpp_p10')
plt.xlabel('budget')
plt.ylabel('$||\Theta_t-\Theta_*||_2$')
plt.legend(fontsize=20)
plt.yscale('log')
plt.show()

In [ ]:
start2 = int(N/p2)
start3 = int(0.25*N/p3)
start4 = int(0.5*N/p4)
start5 = int(0.75*N/p5)

gap2 = int(N/p2)
gap3 = int(N/p3)
gap4 = int(N/p4)
gap5 = int(N/p5)

idx2 = range(start2,maxit2,gap2)
idx2 = np.concatenate((np.array([int(0.25*N/p2)]),idx2))
xidx2 = np.array(range(start2*p2,maxit2*p2,p2*gap2))
xidx2 = np.concatenate((np.array([int(0.25*N)]),xidx2))
yidx2 = grad2.mean(1)[idx2]
zidx2 = err2.mean(1)[idx2]
grad_std_idx2 = np.std(grad2, axis=1)[idx2]
err_std_idx2 = np.std(err2, axis=1)[idx2]

idx4 = range(start4,maxit4+start4-int(N/p4)+gap4,gap4)
idx4 = np.concatenate((np.array([int(0.25*N/p4)]),idx4))
xidx4 = np.array(range(start4*p4,maxit4*p4+start4*p4-N+p4*gap4,p4*gap4))
xidx4 = np.concatenate((np.array([int(0.25*N)]),xidx4))
yidx4 = grad4.mean(1)[idx4]
zidx4 = err4.mean(1)[idx4]
grad_std_idx4 = np.std(grad4, axis=1)[idx4]
err_std_idx4 = np.std(err4, axis=1)[idx4]

idx3 = range(start3,maxit3+start3-int(N/p3)+gap3,gap3)
idx3 = np.concatenate((np.array([int(0.25*N/p3)]),idx3))
xidx3 = np.array(range(start3*p3,maxit3*p3+start3*p3-N+p3*gap3,p3*gap3))
xidx3 = np.concatenate((np.array([int(0.25*N)]),xidx3))
yidx3 = grad3.mean(1)[idx3]
zidx3 = err3.mean(1)[idx3]
grad_std_idx3 = np.std(grad3, axis=1)[idx3]
err_std_idx3 = np.std(err3, axis=1)[idx3]

idx5 = range(start5,maxit5+start5-int(N/p5)+gap5,gap5)
idx5 = np.concatenate((np.array([int(0.25*N/p5)]),idx5))
xidx5 = np.array(range(start5*p5,maxit5*p5+start5*p5-N+p3*gap5,p3*gap5))
xidx5 = np.concatenate((np.array([int(0.25*N)]),xidx5))
yidx5 = grad5.mean(1)[idx5]
zidx5 = err5.mean(1)[idx5]
grad_std_idx5 = np.std(grad5, axis=1)[idx5]
err_std_idx5 = np.std(err5, axis=1)[idx5]

In [ ]:
plt.rcParams.update({'font.size': 20})
fig, ax = plt.subplots()
ax.errorbar(xidx2, yidx2, yerr=grad_std_idx2/np.sqrt(trial_total), fmt='r-', label='iid_p5')
ax.errorbar(xidx3, yidx3, yerr=grad_std_idx3/np.sqrt(trial_total), fmt='b-', label='iid_p10')
ax.errorbar(xidx4, yidx4, yerr=grad_std_idx4/np.sqrt(trial_total), fmt='r--', label='dpp_p5')
ax.errorbar(xidx5, yidx5, yerr=grad_std_idx5/np.sqrt(trial_total), fmt='b--', label='dpp_p10')
ax.legend()
plt.xlabel('budget')
plt.ylabel('norm of gradient')
plt.legend(fontsize=20)
plt.show()

plt.rcParams.update({'font.size': 20})
fig, ax = plt.subplots()
ax.errorbar(xidx2, zidx2, yerr=err_std_idx2/np.sqrt(trial_total), fmt='r-', label='iid_p5')
ax.errorbar(xidx3, zidx3, yerr=err_std_idx3/np.sqrt(trial_total), fmt='b-', label='iid_p10')
ax.errorbar(xidx4, zidx4, yerr=err_std_idx4/np.sqrt(trial_total), fmt='r--', label='dpp_p5')
ax.errorbar(xidx5, zidx5, yerr=err_std_idx5/np.sqrt(trial_total), fmt='b--', label='dpp_p10')
ax.legend()
plt.xlabel('budget')
plt.ylabel('$||\Theta_t-\Theta_*||_2$')
plt.legend(fontsize=20)
plt.show()